In [1]:
from langchain_openai import ChatOpenAI, ChatOpenAI
from langchain.text_splitter import CharacterTextSplitter
from langchain_core.documents import Document

from langchain_openai import OpenAIEmbeddings


In [2]:
import pandas as pd
docs = []
for i in range(100):
   doc = Document(page_content=f"Hello, world! {i}")
   doc.metadata = {"source": "test", "page": i}
   docs.append(doc)

docs_dict = [{p:q for (p,q) in x} for x in docs]
pd.DataFrame(docs_dict)

,id,metadata,page_content,type
0,None,"{'source': 'test', 'page': 0}","Hello, world! 0",Document
1,None,"{'source': 'test', 'page': 1}","Hello, world! 1",Document
2,None,"{'source': 'test', 'page': 2}","Hello, world! 2",Document
3,None,"{'source': 'test', 'page': 3}","Hello, world! 3",Document
4,None,"{'source': 'test', 'page': 4}","Hello, world! 4",Document
...,...,...,...,...
95,None,"{'source': 'test', 'page': 95}","Hello, world! 95",Document
96,None,"{'source': 'test', 'page': 96}","Hello, world! 96",Document
97,None,"{'source': 'test', 'page': 97}","Hello, world! 97",Document
98,None,"{'source': 'test', 'page': 98}","Hello, world! 98",Document


In [3]:
import os
folder_path = r"C:\Users\Sese\autosave\meditalk\html_files"

docs = []

from langchain_community.document_loaders import UnstructuredHTMLLoader

for idx, file_path in enumerate(os.listdir(folder_path)):
   file_path = os.path.join(folder_path, file_path)
   loader = UnstructuredHTMLLoader(file_path)
   data = loader.load()
   
   chap = data[0].page_content.split('\n')[0].split('###')[0]
   remove_first_line = '\n'.join(data[0].page_content.split('\n')[2:])
   data[0].page_content = f"단원명: {chap}\n\n{remove_first_line}"
   docs.append(data[0])
   if idx <= 3:
      print(data[0].page_content)
      print('='*100)
      print('='*100)


단원명: 전공의 소아과

(2850, '테스트', 'basic', 1)

테스트222

(2851, 'ㅁㄴㅇㄹ', 'basic', 1)

34ㄹ222
단원명: 전공의 산부인과

(2848, '임산부의 영상검사 ', 'basic', 1)

초음파와 MRI: 초음파와 MRI와 관련된 태아 위해는 보고된 바 없다. 따라서 임산부에서 가장 먼저 고려하는 영상검사이다.

조영제: 임산부에 있어 가돌리늄 조영제는 꼭 필요한 경우에만 사용되어야 한다.

가돌리늄 MRI 와 MRI 촬영이 없었던 임신을 비교했을 때, 류마티스, 염증성 및 피부질환을 앓은 신생아가 유의하게 더 많게 보고된 코호트 연구가 있어, 위해와 이익을 충분히 비교해본 후에 시행할 것을 권고한다.

X ray, CT: 방사선 피폭에 대한 우려로 방사선 검사는 되도록 피하는 것이 좋으나, 의학적으로 꼭 필요한 경우엔 시행되어야 한다. 그러나 항상 초음파 및 MRI를 우선적으로 고려해야 한다.

취약 시기: 방사선으로 인한 중추신경계 기형(e.g> 지적장애 등)은 태아 주수 8~15주경에 가장 취약하다고 알려져 있다.

일반적인 teratogen(기형유발물질)의 경우 3~8주경이 가장 취약한 시기이다.

영향: 임신 중 50mGy 이하의 방사선 노출은 태아의 기형이나 유산 확률을 증가시키지 않는다. 임신 중절은 여러 인자의 영향을 받는 개인적인 의사결정이나 100mGy 이하의 태아 선량에 대해선 임신 중절을 고려하는 것이 적절치 않다.



Chest PA의 경우 0.01mGy 미만이며, 대부분의 진단적 목적은 50mGy 에 한참 못 미친다.

물론, 방사선 검사는 최대한 피하는 것이 좋지만, 우연히 임신한 줄 모르고 X-ray를 촬영한 산모가 주어졌을 때, 1회의 진단 목적의 X-ray 검사는 태아에게 해로운 결과를 초래하지 않으므로 이로 인해 임신 중절을 고려하는 건 적절치 않다. 경과관찰, 혹은 해당 주수에 맞는 산전진찰을 시행한다.

방사선 동위원소: 임신 중 치료 목적의 요오드 방사성 동위원소는 금기

In [10]:
import dotenv
from langchain_chroma import Chroma
dotenv.load_dotenv()
embedding = OpenAIEmbeddings(model="text-embedding-3-small")


vectorstore = Chroma(
    collection_name="medical",
    embedding_function=embedding,
    persist_directory="chroma_db"  # <- 폴더는 유지됩니다
)
if vectorstore:
   all_ids = vectorstore.get()['ids']
   if all_ids:
      vectorstore.delete(ids=all_ids)
for i in range(0, len(docs), 100):
   vectorstore.add_documents(docs[i:i+100])



In [23]:
## Embedding documents
query = "충수돌기염의 치료"

retreiver = vectorstore.as_retriever(
   search_kwargs={"k": 10}
)

results = retreiver.invoke(query)
for i in results:
   print(i.page_content)
   print('='*100)


단원명: 소화기(위장관)

(3828, '충수돌기 종양', 'basic', 1)

충수돌기에서 발생할 수 있는 종양성 질환은 다음과 같다.

충수돌기 신경내분비종양 (Neuroendocrine neoplasm, carcinoid tumor)

충수돌기 선암종 (adenocarcinoma)

점액성 종양 (Mucinous appendiceal neoplasms)

(3830, '충수돌기 신경내분비종양 (ANEN, carcinoid tumor)', 'basic', 2)

기원: enterochromaffin cell

특징

serotonin, substance P, neurokinin 등의 호르몬 분비 종양

대부분 무증상, 간전이 발생시 carcinoid syndrome (홍조, 복통, 설사, 쌕쌕거림 등) 발생

대부분 충수돌기염으로 절제한 후 조직검사 시행시 우연히 발견

치료 (충수돌기절제술 후)

<1cm, 주위 림프절 전이 없음: 경과관찰

1~2cm, lymphovascular invasion 또는 mesoappendix 침범: Rt. hemicolectomy

>2cm: Rt hemicolectomy

(3829, '충수돌기 선암종 (appendiceal adenocarcinoma)', 'basic', 3)

기원: glandular epithelial cell (colonic type adenoCA)

대부분 충수돌기염으로 절제 후 조직검사 시행시 우연히 발견

치료: Rt hemicolectomy & 주위 림프절 절제술 → 보조항암치료

(3831, '충수돌기 점액성 종양 (MAN)', 'basic', 4)

특징

대부분 충수돌기염으로 절제 후 조직검사 시행시 우연히 발견

복강 내에서 파열된 경우 intraperitoneal spread를 통한 복막가성점액종 (pseudomyxoma peritonei) 발생 가능

종류 및 처치

low grade (benign mucocele)

<2cm, margin (-), base/mesoappend